In [ ]:
# This R environment comes with all of CRAN preinstalled, as well as many other helpful packages
# The environment is defined by the kaggle/rstats docker image: https://github.com/kaggle/docker-rstats
# For example, here's several helpful packages to load in 

library(ggplot2) # Data visualization
library(readr) # CSV file I/O, e.g. the read_csv function
library(tseries)
library(forecast)

system("ls ../input")

# Any results you write to the current directory are saved as output.

**Loading the data**

In [ ]:
load("../input/elecequip.rda")

In [ ]:
elecequip

In [ ]:
train <- ts(elecequip,frequency = 12,start = c(1996,1), end=c(2000,12))

In [ ]:
valid <- window(elecequip,start = c(2001,1), end=c(2005,12),frequency = 12)

In [ ]:
train

In [ ]:
valid

In [ ]:
frequency(elecequip)

In [ ]:
plot((elecequip))

In [ ]:
plot(log(elecequip))

In [ ]:
# trend estimation using linear regression
reg=lm(log(elecequip)~time(elecequip))
plot(log(elecequip))
abline(reg)

In [ ]:
elecequip_diff=diff(diff(log(elecequip),lag =12), differences =1)
plot(elecequip_diff)

In [ ]:
# Augmented Dickey-Fuller Test for stationarity
adf.test(elecequip_diff, alternative="stationary", k=0)

In [ ]:
elecequip

In [ ]:
k=0
x<-ts(elecequip,frequency=12, start=c(1996,1),end=c(2000,12))
x<-tsclean(x,replace.missing=TRUE, lambda='auto')
x_de<-stl(x, "periodic")
x1<-x-x_de$time.series[,2]
fit<-auto.arima(x1)
x_pred<-forecast(fit,3, level = 90)
x_real<-x_pred$mean+x_de$time.series[60+k,2]
write(x_real,file="out_mean_0.csv",append=TRUE,sep=",")
write(x-x_de$time.series[,2]-x_pred$fitted,file="out_error_0.csv",ncolumns=1,sep=",")


k=0
for (j in 1:5)
{
    k<-k+1
    x<-ts(elecequip,frequency=12, start=c(1996,1),end=c(2001,j))
    x<-tsclean(x,replace.missing=TRUE, lambda='auto')
    x_de<-stl(x, "periodic")
    x1<-x-x_de$time.series[,2]
    fit<-auto.arima(x1)
    x_pred<-forecast(fit,3, level = 90)
    x_real<-x_pred$mean+x_de$time.series[60+k,2]
    write(x_real,file=paste("out_mean_",j,'.csv',sep=''),append=TRUE,sep=",")
    write(x-x_de$time.series[,2]-x_pred$fitted,file=paste("out_error",j,'.csv',sep=''),ncolumns=1,sep=",")
}
